In [1]:
import os,sys
import numpy as np
# xarray
import xarray as xr

import multiprocessing.popen_spawn_posix
from distributed import Client
# from dask.distributed import Client
c = Client()
c

Client Scheduler: tcp://127.0.0.1:34521 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 322.12 GB


In [2]:
prefix = "eORCA025.L75-IMHOTEP"
fo="1d" 
decomposition = "GREENLAND"
diridat="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+"."
diro = "/gpfsscratch/rech/cli/uor98hu/Bilans_4_fig6/"+decomposition+"/"

chunk_size = {"time_counter":10,"x":200,"y":320}

In [ ]:
%%time
for year in np.arange(1980,1981):
    y1 = str(year)
    print(y1)
    # create directory for the year
    diroyear = y1+"-concat"
    diropath = os.path.join(diro, diroyear)
    # os.mkdir(diropath)
    i = 0
    for month in ['01']: #,'02','03','04','05','06','07','08','09','10','11','12']: # 
        # print("month",month)

        ######## ---------- load GAI
        nexp1 = "GAI"
        # print(nexp)

        fileT_exp1 = diridat+nexp1+"-S/"+"/1d/"+y1+"-concat"+"/"+prefix+"."+nexp1+"_y"+y1+"m"+month+"_1d_gridT.nc"
        fileU_exp1 = diridat+nexp1+"-S/"+"/1d/"+y1+"-concat"+"/"+prefix+"."+nexp1+"_y"+y1+"m"+month+"_1d_gridU.nc"
        fileV_exp1 = diridat+nexp1+"-S/"+"/1d/"+y1+"-concat"+"/"+prefix+"."+nexp1+"_y"+y1+"m"+month+"_1d_gridV.nc"
        
        gridT_exp1 = xr.open_dataset(fileT_exp1,chunks = chunk_size).squeeze()
        gridU_exp1 = xr.open_dataset(fileU_exp1,chunks = chunk_size).squeeze()
        gridV_exp1 = xr.open_dataset(fileV_exp1,chunks = chunk_size).squeeze()

        S_exp1 = gridT_exp1.vosaline
        U_exp1 = gridU_exp1.vozocrtx
        V_exp1 = gridV_exp1.vomecrty
    
        ######## ---------- load AI
        nexp2 = "AI"
        # print(nexp)

        fileT_exp2 = diridat+nexp2+"-S/"+"/1d/"+y1+"-concat"+"/"+prefix+"."+nexp2+"_y"+y1+"m"+month+"_1d_gridT.nc"
        fileU_exp2 = diridat+nexp2+"-S/"+"/1d/"+y1+"-concat"+"/"+prefix+"."+nexp2+"_y"+y1+"m"+month+"_1d_gridU.nc"
        fileV_exp2 = diridat+nexp2+"-S/"+"/1d/"+y1+"-concat"+"/"+prefix+"."+nexp2+"_y"+y1+"m"+month+"_1d_gridV.nc"
        
        gridT_exp2 = xr.open_dataset(fileT_exp2,chunks = chunk_size).squeeze()
        # gridU_exp2 = xr.open_dataset(fileU_exp2,chunks = chunk_size).squeeze()
        # gridV_exp2 = xr.open_dataset(fileV_exp2,chunks = chunk_size).squeeze()

        S_exp2 = gridT_exp2.vosaline
#         U_exp2 = gridU_exp2.vozocrtx
#         V_exp2 = gridV_exp2.vomecrty
        
        ####### --------------- compute Greenland's terms
        S_G = S_exp1 - S_exp2
        # U_G = U_exp1 - U_exp2
        # V_G = V_exp1 - V_exp2
        
        ####### ------------- assign the new variables to the dataset
        # gridT_exp1_changed = gridT_exp1.assign(vosaline=S_G)
        gridT_exp1_changed = gridT_exp1.copy(data={"vosaline": S_G, 'votemper':gridT_exp1.votemper, 'time_counter_bounds':gridT_exp1.time_counter_bounds,\
                                                   'deptht_bounds':gridT_exp1.deptht_bounds, 'time_centered_bounds':gridT_exp1.time_centered_bounds, 'e3t':gridT_exp1.e3t})
        # gridU_exp1_changed = gridU_exp1.assign(vozocrtx=U_G)
        # gridV_exp1_changed = gridV_exp1.assign(vomecrty=V_G)
        
        print("saving gridT")
        # gridT_exp1_changed.to_netcdf(diropath+"/eORCA025.L75-IMHOTEP."+decomposition+"_y"+y1+"m"+month+"_1d_gridT.nc", mode='w')
        gridT_exp1_changed.to_netcdf(diropath+"/eORCA025.L75-IMHOTEP."+decomposition+"_y"+y1+"m"+month+"_1d_gridT_v2.nc", mode='w')

        # print("saving gridU")
        # gridU_exp1_changed.to_netcdf(diropath+"/eORCA025.L75-IMHOTEP."+decomposition+"_y"+y1+"m"+month+"_1d_gridU.nc", mode='w')
        # print("saving gridV")
        # gridV_exp1_changed.to_netcdf(diropath+"/eORCA025.L75-IMHOTEP."+decomposition+"_y"+y1+"m"+month+"_1d_gridV.nc", mode='w')


1980
saving gridT


In [13]:
gridT_exp1.e3t[0,0,500,500].values

array(1.02393591)

In [14]:
xr.open_dataset(diropath+"/eORCA025.L75-IMHOTEP."+decomposition+"_y"+y1+"m"+month+"_1d_gridT.nc").e3t[0,0,500,500].values


array(1.02393591)